In [6]:
# import actions 
# project_id  = 27
# active_scenes = actions.get_active_scenes(project_id)
# # print(active_scenes)
# scenes_words_count_list = []
# for scene in active_scenes:
#     scene_text = scene.get("scene_text")
#     no_words_in_scene = len(scene_text.split())
#     scenes_words_count_list.append(no_words_in_scene)

# print(scenes_words_count_list)
# total_words_in_project = sum(scenes_words_count_list)
# print(total_words_in_project)



# import funcs_supabase
# words_list = funcs_supabase.select_data("video_creator_projects", "id", project_id)[0].get("words_list")
# print(len(words_list))


In [20]:
import actions
import funcs_supabase

def scene_words_len_match(project_id):
    active_scenes = actions.get_active_scenes(project_id)
    scene_text_list = []
    for scene in active_scenes:
        scene_text = scene.get("scene_text")
        # print(scene_text)
        scene_split = scene_text.split()
        # print(scene_split)
        scene_text_list = scene_text_list + scene_split
    # print(scene_text_list)
    # print(len(scene_text_list))

    words_list = funcs_supabase.select_data("video_creator_projects", "id", project_id)[0].get("words_list")
    # print(words_list)
    # Extract words that contain letters or numbers
    words = [item['word'] for item in words_list if any(char.isalnum() for char in item['word'])]
    # print(words)
    # print(len(words))

    if len(scene_text_list) == len(words):
        return True
    else:
        return False
    





2023-11-01 18:00:34,263:INFO - HTTP Request: GET https://fpyltvtkpkrkzortucoa.supabase.co/rest/v1/scenes?select=%2A&project_id=eq.27&status=eq.Active "HTTP/1.1 200 OK"


['One', 'day', 'there', 'was', 'a', 'cute', 'dog', 'called', 'Dorris.', 'she', 'went', 'out', 'to', 'play', 'in', 'the', 'rain.', 'At', 'first,', 'she', "didn't", 'like', 'the', 'rain.', 'Then', 'she', 'overcame', 'her', 'fear.', 'Now', 'she', 'is', 'always', 'muddy!']
34


2023-11-01 18:00:34,476:INFO - HTTP Request: GET https://fpyltvtkpkrkzortucoa.supabase.co/rest/v1/video_creator_projects?select=%2A&id=eq.27 "HTTP/1.1 200 OK"


[{'end': 0.188, 'word': 'One', 'score': 0.716, 'start': 0.088}, {'end': 0.428, 'word': 'day', 'score': 0.83, 'start': 0.228}, {'end': 0.628, 'word': 'there', 'score': 0.924, 'start': 0.488}, {'end': 0.768, 'word': 'was', 'score': 0.78, 'start': 0.668}, {'end': 0.848, 'word': 'a', 'score': 1, 'start': 0.828}, {'end': 1.169, 'word': 'cute', 'score': 0.732, 'start': 0.908}, {'end': 1.529, 'word': 'dog', 'score': 0.86, 'start': 1.209}, {'end': 1.809, 'word': 'called', 'score': 0.778, 'start': 1.569}, {'end': 2.309, 'word': 'Doris.', 'score': 0.881, 'start': 1.869}, {'end': 2.929, 'word': 'She', 'score': 0.983, 'start': 2.849}, {'end': 3.089, 'word': 'went', 'score': 1, 'start': 2.969}, {'end': 3.249, 'word': 'out', 'score': 0.85, 'start': 3.149}, {'end': 3.37, 'word': 'to', 'score': 0.75, 'start': 3.29}, {'end': 3.65, 'word': 'play', 'score': 0.872, 'start': 3.41}, {'end': 3.75, 'word': 'in', 'score': 0.829, 'start': 3.67}, {'end': 3.85, 'word': 'the', 'score': 0.834, 'start': 3.77}, {'end

In [7]:

import actions
import funcs_supabase

def process_project_scenes(project_id):

    scene_timings_raw = []

    active_scenes = actions.get_active_scenes(project_id)

    # Check if active_scenes is None or empty before proceeding
    if not active_scenes:
        return "No active scenes available for this project."

    total_words_in_project = 0

    for scene_counter, scene in enumerate(active_scenes):
        scene_text = scene.get("scene_text", "")
        scene_id = scene.get("id")

        # If scene_text or scene_id is missing, skip to next iteration
        if not scene_text or scene_id is None:
            continue

        # print(scene_text)
        words_in_scene = scene_text.split()
        no_words_in_scene = len(words_in_scene)

        # print(f"scene has {no_words_in_scene} words")
        # print(f"the first word is {words_in_scene[0]}")
        # print(f"the last word is {words_in_scene[-1]}")

        total_words_in_project += no_words_in_scene
        first_word_location = total_words_in_project - no_words_in_scene
        last_word_location = total_words_in_project - 1

        # print(f"first word location: {first_word_location}")
        # print(f"last word location: {last_word_location}")

        words_list = funcs_supabase.select_data("video_creator_projects", "id", project_id)[0].get("words_list")

        try:
            first_word_from_words_list = words_list[first_word_location]
            last_word_from_words_list = words_list[last_word_location]
        except IndexError:
            # Handle this error if word locations are out of bounds
            continue

        # print(f"first word from words list: {first_word_from_words_list}")
        # print(f"last word from words list: {last_word_from_words_list}")

        if scene_counter == 0:
            words_list[first_word_location]["start"] = 0

        scene_length = last_word_from_words_list["end"] - first_word_from_words_list["end"]
        scene_length = round(scene_length, 3)

        data_to_update = {
            "length": scene_length,
            "start": first_word_from_words_list["start"],
            "end": last_word_from_words_list["end"]
        }

        # funcs_supabase.update_data("scenes", scene_id, {"timings": data_to_update})
        # print(f"scene length: {scene_length}")
        # print("\n")

        scene_timings_raw.append(data_to_update)

    return scene_timings_raw
    
    



In [8]:
import os
from pydub import AudioSegment
import requests

def get_audio_duration(project_id):
    # Assume funcs_supabase.select_data() returns the necessary data
    audio_file_url = funcs_supabase.select_data("video_creator_projects", "id", project_id)[0].get("speech_audio")

    # Specify the folder where the audio file will be downloaded
    download_folder = "downloaded_audio"
    os.makedirs(download_folder, exist_ok=True)

    # Extract the filename and create a path for the downloaded file
    filename = os.path.basename(audio_file_url)
    file_path = os.path.join(download_folder, filename)

    # Download the audio file
    response = requests.get(audio_file_url)
    if response.status_code == 200:
        with open(file_path, 'wb') as file:
            file.write(response.content)
    else:
        raise Exception("Failed to download the audio file")

    try:
        # Load the audio file and get its duration
        audio = AudioSegment.from_file(file_path)
        duration_seconds = len(audio) / 1000  # pydub calculates in millisec
    finally:
        # Delete the audio file after processing
        if os.path.exists(file_path):
            os.remove(file_path)

    return duration_seconds


# # Usage example
# duration = get_audio_duration(27)
# print(f"The audio duration is {duration} seconds.")




In [9]:
def process_scene_timings(scene_timings_raw, project_id):
    audio_duration = get_audio_duration(project_id)
    
    scene_timings_processed = []
    for i, scene in enumerate(scene_timings_raw):

        # print(scene)

        # first scene starts at 0
        if i == 0:
            scene["start"] = 0
       
        # if not the first scene, the start time is the end time of the previous scene
        else:
            scene["start"] = scene_timings_raw[i-1]["end"]
        
        # if not the last scene, the end time is the start time of the next scene
        if i < len(scene_timings_raw) - 1:
            scene["end"] = scene_timings_raw[i+1]["start"]


        # if the last scene, end at the end of the audio
        else:
            scene["end"] = audio_duration

        
        
        scene["start"] = round(scene["start"], 3)
        scene["end"] = round(scene["end"], 3)

        # recalculate the length 
        scene["length"] = scene["end"] - scene["start"]

        # round the length to 3 decimal places
        scene["length"] = round(scene["length"], 3)


        scene_timings_processed.append(scene)

    return scene_timings_processed


In [10]:
def update_timings_database(processed_scene_timings, project_id):
    resp = []
    active_scenes = actions.get_active_scenes(project_id)

    if len(active_scenes) != len(processed_scene_timings):
        print("The number of scenes in the database and the number of scenes in the processed scene timings do not match.")
        return
    
    for i, scene in enumerate(processed_scene_timings):

        data_to_update = {
        "length": scene["length"],
        "start": scene["start"],
        "end": scene["end"]
        }

        scene_id = active_scenes[i]["id"]
        print(f"updating scene {scene_id} with {data_to_update}")
        update = funcs_supabase.update_data("scenes", scene_id, {"timings": data_to_update})
        print(update)
        print("\n")
        resp.append(update)

    return resp


In [11]:
# Usage
project_id = 27
scene_timings_raw = process_project_scenes(project_id)
scene_timings_processed = process_scene_timings(scene_timings_raw, project_id)
update = update_timings_database(scene_timings_processed, project_id)

2023-11-01 16:48:18,773:INFO - HTTP Request: GET https://fpyltvtkpkrkzortucoa.supabase.co/rest/v1/scenes?select=%2A&project_id=eq.27&status=eq.Active "HTTP/1.1 200 OK"
2023-11-01 16:48:18,897:INFO - HTTP Request: GET https://fpyltvtkpkrkzortucoa.supabase.co/rest/v1/video_creator_projects?select=%2A&id=eq.27 "HTTP/1.1 200 OK"
2023-11-01 16:48:19,079:INFO - HTTP Request: GET https://fpyltvtkpkrkzortucoa.supabase.co/rest/v1/video_creator_projects?select=%2A&id=eq.27 "HTTP/1.1 200 OK"
2023-11-01 16:48:19,187:INFO - HTTP Request: GET https://fpyltvtkpkrkzortucoa.supabase.co/rest/v1/video_creator_projects?select=%2A&id=eq.27 "HTTP/1.1 200 OK"
2023-11-01 16:48:19,289:INFO - HTTP Request: GET https://fpyltvtkpkrkzortucoa.supabase.co/rest/v1/video_creator_projects?select=%2A&id=eq.27 "HTTP/1.1 200 OK"
2023-11-01 16:48:19,408:INFO - HTTP Request: GET https://fpyltvtkpkrkzortucoa.supabase.co/rest/v1/video_creator_projects?select=%2A&id=eq.27 "HTTP/1.1 200 OK"
2023-11-01 16:48:19,490:INFO - HTTP R

updating scene 207 with {'length': 2.849, 'start': 0, 'end': 2.849}
[{'id': 207, 'created_at': '2023-10-31T22:12:47.813861+00:00', 'project_id': 27, 'scene_order_position': 1, 'scene_text': 'One day there was a cute dog called Dorris.', 'status': 'Active', 'album_id': 221, 'theme_keywords': ['Adorable canine creature', 'Empty dog bed', 'Bright sunny weather', 'Tail wagging excitement', 'Moments of peace'], 'sd_prompts': ['(Small fluffy dog, cute), sitting, house front porch, daytime'], 'final_clip': 849, 'timings': {'end': 2.849, 'start': 0, 'length': 2.849}}]


updating scene 208 with {'length': 1.941, 'start': 2.849, 'end': 4.79}


2023-11-01 16:48:20,152:INFO - HTTP Request: PATCH https://fpyltvtkpkrkzortucoa.supabase.co/rest/v1/scenes?id=eq.208 "HTTP/1.1 200 OK"
2023-11-01 16:48:20,237:INFO - HTTP Request: PATCH https://fpyltvtkpkrkzortucoa.supabase.co/rest/v1/scenes?id=eq.209 "HTTP/1.1 200 OK"


[{'id': 208, 'created_at': '2023-10-31T22:12:47.985469+00:00', 'project_id': 27, 'scene_order_position': 2, 'scene_text': 'she went out to play in the rain.', 'status': 'Active', 'album_id': 222, 'theme_keywords': ['Rain dancing droplets', 'Muddy playfield', 'Joyous bouncing outdoors', 'Rainswept landscape', 'Unleashed freedom'], 'sd_prompts': ['(Small fluffy dog), stepping timidly, rain falling, garden, puddles everywhere'], 'final_clip': 949, 'timings': {'end': 4.79, 'start': 2.849, 'length': 1.941}}]


updating scene 209 with {'length': 2.161, 'start': 4.79, 'end': 6.951}
[{'id': 209, 'created_at': '2023-10-31T22:12:48.187687+00:00', 'project_id': 27, 'scene_order_position': 2, 'scene_text': "At first, she didn't like the rain.", 'status': 'Active', 'album_id': 223, 'theme_keywords': ['Dog shielding under tree', 'Frightened canine expression', 'Distinct dislike', 'Avoidance of rain puddles', 'Wet fur discomfort'], 'sd_prompts': ['(Small fluffy dog), Sitting in the rain, unhappy expr

2023-11-01 16:48:20,383:INFO - HTTP Request: PATCH https://fpyltvtkpkrkzortucoa.supabase.co/rest/v1/scenes?id=eq.210 "HTTP/1.1 200 OK"
2023-11-01 16:48:20,475:INFO - HTTP Request: PATCH https://fpyltvtkpkrkzortucoa.supabase.co/rest/v1/scenes?id=eq.211 "HTTP/1.1 200 OK"


[{'id': 210, 'created_at': '2023-10-31T22:12:48.365005+00:00', 'project_id': 27, 'scene_order_position': 3, 'scene_text': 'Then she overcame her fear.', 'status': 'Active', 'album_id': 224, 'theme_keywords': ['Overcoming obstacles', 'Bravery step into rain', 'Change in perception', 'Fearlessness against the weather', 'Acceptance of change'], 'sd_prompts': ['(Small fluffy dog), closed eyes, shaking off rain, expression of surprise and enjoyment, bright colors'], 'final_clip': 566, 'timings': {'end': 9.172, 'start': 6.951, 'length': 2.221}}]


updating scene 211 with {'length': 1.773, 'start': 9.172, 'end': 10.945}
[{'id': 211, 'created_at': '2023-10-31T22:12:48.589199+00:00', 'project_id': 27, 'scene_order_position': 4, 'scene_text': 'Now she is always muddy!', 'status': 'Active', 'album_id': 225, 'theme_keywords': ['Muddy pawprints', 'Happy dirty dog', 'Everyday is a muddy playday', 'Love for mud', 'Filthy fun canine'], 'sd_prompts': ['(Small fluffy dog), covered in mud, happy expressi